In [208]:
import numpy as np
from IPython.terminal.shortcuts import previous_history_or_previous_completion

In [209]:
import numpy as np

v1 = np.array([[1],
               [2],
               [4]])
v2 = np.array([[1],
               [0],
               [1]])
v3 = np.array([[3],
               [2],
               [6]])
v4 = np.array([[4],
               [4],
               [10]])

# Horizontally stack to form a 3x4 matrix
V = np.hstack((v1, v2, v3, v4))
[D, U] = np.linalg.eigh(V.dot(V.T))
print(D)

[1.81388436e-15 1.51119465e+00 2.02488805e+02]


In [210]:
u1 = np.reshape(U[:,1], (3,1))
u2 = np.reshape(U[:,2], (3,1))
u = np.hstack((u1, u2))
print(u.shape)

(3, 2)


In [211]:
y = np.array([[7],[10],[22]])
sol = np.linalg.inv(u.T@u)@u.T@y
print(sol)

[[  2.48032208]
 [-25.03693277]]


In [212]:
# Solving for a matrix given its eigenvectors (V) and eigenvalues(e) using formula V @ E @ V.T = A. 
# Note, eigenvalues must be transformed from vector e to matrix E
A = np.array([[2,1],[1,2]])
e, V = np.linalg.eigh(A)
E = np.diagflat(e)
print(E)
print(V @ E @ V.T)

[[1. 0.]
 [0. 3.]]
[[2. 1.]
 [1. 2.]]


In [213]:
# # Original 2D vectors
x1 = np.array([[1],
               [2]])
x2 = np.array([[4],
               [1]])
x3 = np.array([[3],
               [3]])
x4 = np.array([[5],
               [2]])
x5 = np.array([[1],
               [1]])

phi = np.array([[1, 2, 1],
                [4, 1, 1],
                [3, 3, 1],
                [5, 2, 1],
                [1, 1, 1]])

int_op = 1 / 2 * phi.T @ phi
print("Integral Operator Matrix:  \n", int_op)

# To find the eigenfunctions of the RKHS, find the necessary eigenvectors of the integral operator matrix
[e, v] = np.linalg.eigh(int_op)
print("\neigenvals:")
print(e)
print("\neigenvecs:")
print(v)

Integral Operator Matrix:  
 [[26.  13.   7. ]
 [13.   9.5  4.5]
 [ 7.   4.5  2.5]]

eigenvals:
[ 0.24980145  2.49830389 35.25189466]

eigenvecs:
[[-0.08471245 -0.52421445 -0.84736238]
 [-0.33709272  0.81535207 -0.4707117 ]
 [ 0.93765255  0.24576454 -0.24577935]]


In [214]:
# vec, the eigenvectors of the integral operator, is the eigenfunction matrix, and any point in the RKHS is a linear combination of it
y = np.array([[1],[2],[1]])
w = np.linalg.inv(vec.T@vec)@vec.T@y
print(w, "\n")
print(vec@w)

[[ 0.17875466]
 [ 1.35225423]
 [-2.03456513]] 

[[1.]
 [2.]
 [1.]]


In [215]:
# Calculating Kernel Matrix
X = np.array([[1,1],
              [2,2],
              [3,3]])
def k(x,y):
    return ((x.T @ y) + 1)**2
K_1 = np.zeros((3,3))
for i in range(X.shape[0]):
    for j in range(X.shape[0]):
        element = k(X[i,:], X[j,:])
        K_1[i,j] = element
print("First way of getting Kernel Matrix using kernel function:\n ", K_1)

First way of getting Kernel Matrix using kernel function:
  [[  9.  25.  49.]
 [ 25.  81. 169.]
 [ 49. 169. 361.]]


In [216]:
# Calculating Kernel Matrix
def phi(x):
    phi = np.ones((1,6))
    phi[:,0] = x[0] ** 2
    phi[:,1] = np.sqrt(2) * x[0] * x[1]
    phi[:,2] = np.sqrt(2) * x[0]
    phi[:,3] = np.sqrt(2) * x[1]
    phi[:,4] = x[1] ** 2
    phi[:,5] = 1
    return phi
phi_1 = phi(X[0,:])
phi_2 = phi(X[1,:])
phi_3 = phi(X[2,:])
phi = np.vstack([phi_1,phi_2,phi_3])

K_2 = phi@phi.T
print("Second way of getting kernel matrix using feature map:\n ", K_2)

Second way of getting kernel matrix using feature map:
  [[  9.  25.  49.]
 [ 25.  81. 169.]
 [ 49. 169. 361.]]


In [217]:
# Calculating Integral Operator Matrix
def intop_with_phi(X, phi):
    # This function makes an integral operator matrix when you know the feature map
    # X (train matrix) has shape (n,d). The n is the number of rows.
    n = X.shape[0]
    T = (1/n) * phi.T @ phi
    return T
T = intop_with_phi(X, phi)

sig, v = np.linalg.eig(T)
print("hello")
def nonzero_eigenvectors(matrix):
    tol=1e-10
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    nonzero_mask = np.abs(eigenvalues) > tol  # Avoid numerical precision issues
    return eigenvectors[:, nonzero_mask]

print("eigenvalues of integral operator matrix: ", sig)
print("nonzero eigenvectors: \n", nonzero_eigenvectors(T))

hello
eigenvalues of integral operator matrix:  [ 1.49117396e+02  1.20272082e+00  1.32167011e-02 -1.09600992e-14
 -2.78659036e-15 -5.46063058e-20]
nonzero eigenvectors: 
 [[-0.46780608 -0.16662765  0.05824683]
 [-0.66157771 -0.23564708  0.08237346]
 [-0.24491192  0.56756935 -0.34334121]
 [-0.24491192  0.56756935 -0.34334121]
 [-0.46780608 -0.16662765  0.05824683]
 [-0.06830941  0.49464226  0.86640802]]


In [218]:
sig_T = sig
sig_K, v_K = np.linalg.eig(K_2)

nonzero_values_T = sig_T[0:3]
print("Eigenvalues of integral operator matrix:\n", nonzero_values_T)
print("Eigenvalues of Kernel matrix:\n", sig_K)

factor = sig_K/nonzero_values_T
print("scalar factor between eigenvalues: ", factor)

eigenfunctions = nonzero_eigenvectors(K_2)


Eigenvalues of integral operator matrix:
 [1.49117396e+02 1.20272082e+00 1.32167011e-02]
Eigenvalues of Kernel matrix:
 [4.47352187e+02 3.60816247e+00 3.96501033e-02]
scalar factor between eigenvalues:  [3. 3. 3.]


In [219]:
eigenlist = []
for i in range(sig_K.shape[0]):
    ei = (1/(np.sqrt(sig_K[i]))) * phi.T @ eigenfunctions[i]
    eigenlist.append(ei)
phi_hat = np.stack(eigenlist).T

[[ 4.18421532e-02  5.91737405e-02  6.66243630e-03  6.66243630e-03
   4.18421532e-02 -3.70159893e-03]
 [-1.59933854e+00 -2.26180625e+00 -9.14375068e-01 -9.14375068e-01
  -1.59933854e+00 -2.96964967e-01]
 [ 4.70147281e+00  6.64888662e+00  1.37106629e+00  1.37106629e+00
   4.70147281e+00 -4.76601071e-01]]


In [221]:
def check_subspace_equivalence(phi_hat, phi_T, tol=1e-12):
    """
    Checks whether columns of phi_hat and phi_T span the same subspace.
    
    Parameters:
        phi_hat : np.ndarray of shape (N, k)
            Eigenfunctions (as columns) from the kernel matrix approach.
        phi_T   : np.ndarray of shape (N, k)
            Eigenfunctions (as columns) from the integral operator approach.
        tol     : float
            Tolerance for considering two projectors "equal."
    """
    # 1. Orthonormalize the columns of phi_hat
    #    (QR decomposition automatically produces orthonormal columns in Q)
    Q1, _ = np.linalg.qr(phi_hat)
    
    # 2. Orthonormalize the columns of phi_T
    Q2, _ = np.linalg.qr(phi_T)
    
    # 3. Construct the orthogonal projectors onto each subspace
    #    P1 = Q1 Q1^T, P2 = Q2 Q2^T
    P1 = Q1 @ Q1.T
    P2 = Q2 @ Q2.T
    
    # 4. Compare the difference in Frobenius norm or spectral norm
    diff_norm = np.linalg.norm(P1 - P2, ord='fro')
    
    if diff_norm < tol:
        print(f"Subspace check: PASSED (difference in projectors = {diff_norm:.2e} < tol)")
    else:
        print(f"Subspace check: FAILED (difference in projectors = {diff_norm:.2e} >= tol)")
        
        
def check_pairwise_alignment(phi_hat, phi_T, tol=1e-12):
    """
    Checks if each corresponding column in phi_hat and phi_T is collinear
    (i.e., same up to a scalar) under a certain tolerance.
    
    Parameters:
        phi_hat : np.ndarray of shape (N, k)
        phi_T   : np.ndarray of shape (N, k)
        tol     : float
            Tolerance for "close enough" in vector norms.
    """
    k = phi_hat.shape[1]
    if phi_T.shape[1] != k:
        raise ValueError("phi_hat and phi_T must have the same number of columns")

    for i in range(k):
        # Column i
        v1 = phi_hat[:, i]
        v2 = phi_T[:, i]
        
        # Best scalar to map v2 to v1 in least-squares sense
        # alpha = (v1^T v2) / (v2^T v2)
        denom = np.dot(v2, v2)
        if abs(denom) < tol:
            print(f"Column {i}: v2 is nearly zero vector?")
            continue
        alpha = np.dot(v1, v2) / denom
        
        # Compare alpha * v2 to v1
        diff = v1 - alpha * v2
        diff_norm = np.linalg.norm(diff)
        v1_norm = np.linalg.norm(v1)
        
        if v1_norm > tol:
            rel_err = diff_norm / v1_norm
        else:
            # If v1 is close to zero, just use abs difference
            rel_err = diff_norm
        
        print(f"Column {i}: best alpha = {alpha:.4g}, relative error = {rel_err:.2e}")
        
        if rel_err < tol:
            print(f"  -> Columns match (up to scalar).")
        else:
            print(f"  -> Columns do NOT match within tol.")

# Suppose you have:
# phi_hat: (N, k) from your kernel approach
# phi_T:   (N, k) from your integral operator approach

# 1) Subspace check
check_subspace_equivalence(phi_hat, nonzero_eigenvectors(T), tol=1e-8)

# 2) Pairwise check (only if you've matched columns or there's no degeneracy)
check_pairwise_alignment(phi_hat, nonzero_eigenvectors(T), tol=1e-8)

Subspace check: PASSED (difference in projectors = 1.81e-13 < tol)
Column 0: best alpha = -0.08131, relative error = 2.64e-01
  -> Columns do NOT match within tol.
Column 1: best alpha = -0.1189, relative error = 9.99e-01
  -> Columns do NOT match within tol.
Column 2: best alpha = -0.259, relative error = 1.00e+00
  -> Columns do NOT match within tol.
